In [2]:
from pycsp3 import *
import pandas as pd
from mido import MidiFile
import itertools


df = pd.read_csv('dataset/commu_meta_with_duration.csv')

In [3]:
def reduce_inst(inst_id):
    if "-" in inst_id:
        return inst_id[:inst_id.index("-")]
    else:
        return inst_id

instruments = set([reduce_inst(inst) for inst in df.inst.unique()])

print(f"Instruments reduced from {len(df.inst.unique())} to {len(instruments)}")

Instruments reduced from 130 to 38


In [36]:
df.time_signature.unique()

array(['4/4', '6/8', '3/4'], dtype=object)

In [37]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'audio_key',
       'chord_progressions', 'pitch_range', 'num_measures', 'bpm', 'genre',
       'track_role', 'inst', 'sample_rhythm', 'time_signature', 'min_velocity',
       'max_velocity', 'split_data', 'id', 'duration'],
      dtype='object')

In [38]:
df.track_role.unique()

array(['main_melody', 'accompaniment', 'riff', 'pad', 'sub_melody',
       'bass'], dtype=object)

In [39]:
len(df)

11144

In [40]:
df[["time_signature", "id"]].groupby("time_signature").count()

,id
time_signature,
3/4,1269
4/4,9299
6/8,576


In [41]:
# Possibly filter rows by bpm, genre etc.
df = df[df["time_signature"]=="4/4"]

df[["track_role", "id"]].groupby("track_role").count()

,id
track_role,
accompaniment,2016
bass,403
main_melody,2404
pad,1635
riff,1022
sub_melody,1819


In [42]:
df[["num_measures", "id"]].groupby("num_measures").count()

,id
num_measures,
4,2609
5,2
8,6579
9,64
16,44
17,1


In [43]:

# Parameters

# Tracks must have a number of samples equals to 2/4/8 (usually a multiple of 4)
number_of_samples_per_track = 4

# Sample can be repeated 0, 1 or 2 times in a segment to match the number of measures of the samples in the other tracks
sample_repetitions = {0, 1, 2}

num_measures = [int(m) for m in df.num_measures.unique()]

In [57]:
instruments = list(instruments)

def to_inst_id(instrument):
    return instruments.index(reduce_inst(instrument)) + 1

def get_domain_from_track_role(df, track_role):
    result = list(df[df["track_role"]==track_role].id.unique())
    #result.append(f"{track_role}_padding")
    return result

# Create commu_id to num_measures correspondence
def get_num_measures_correspondence(df, track_role):
    
    commu_id_to_num_measures = set()

    for index, row in df[df["track_role"]==track_role].iterrows():
        commu_id_to_num_measures.add((row["id"], int(row["num_measures"])))
        
    return commu_id_to_num_measures

def get_num_measures_array(df, ids):
    result = [None] * len(ids)
    for index, commu_id in enumerate(ids):
        #if commu_id.endswith("_padding"):
        #    # to check
        #    result[index] = 4
        #else:    
        result[index] = int(df[df["id"]==commu_id].num_measures)
    return result

def get_instrument_array(df, ids):
    result = [None] * len(ids)
    for index, commu_id in enumerate(ids):
        result[index] = to_inst_id(str(df[df["id"]==commu_id].inst.values[0]))
    return result

In [58]:
main_melody_ids = get_domain_from_track_role(df, "main_melody")
main_melody_num_measures = get_num_measures_array(df, main_melody_ids)
main_melody_instruments = get_instrument_array(df, main_melody_ids)

accompaniment_ids = get_domain_from_track_role(df, "accompaniment")
accompaniment_num_measures = get_num_measures_array(df, accompaniment_ids)
accompaniment_instruments = get_instrument_array(df, accompaniment_ids)

riff_ids = get_domain_from_track_role(df, "riff")
riff_num_measures = get_num_measures_array(df, riff_ids)
riff_instruments = get_instrument_array(df, riff_ids)

pad_ids = get_domain_from_track_role(df, "pad")
pad_num_measures = get_num_measures_array(df, pad_ids)
pad_instruments = get_instrument_array(df, pad_ids)

sub_melody_ids = get_domain_from_track_role(df, "sub_melody")
sub_melody_num_measures = get_num_measures_array(df, sub_melody_ids)
sub_melody_instruments = get_instrument_array(df, sub_melody_ids)

bass_ids = get_domain_from_track_role(df, "bass")
bass_num_measures = get_num_measures_array(df, bass_ids)
bass_instruments = get_instrument_array(df, bass_ids)

In [59]:
clear()

# For each track role we have an array of variables of number_of_samples_per_track size
# The domain of the variables is the set of samples with the corresponding track_role (actually the identifiers of the samples)
#main_melody = VarArray(size=number_of_samples_per_track, dom=get_domain_from_track_role(df, "main_melody")) 
main_melody = VarArray(size=[number_of_samples_per_track, len(main_melody_ids)], dom={0, 1})

# For each track role we have an array variables of number_of_samples_per_track size
# Each variable needs to be bound to the number of measures of the sample 
#main_melody_measures = VarArray(size=[number_of_samples_per_track, len(main_melody_ids)], dom=num_measures) 

# main_melody_repetitions[i] is the number of times the sample must be repeated in a sample
main_melody_repetitions = VarArray(size=[number_of_samples_per_track, len(main_melody_ids)], dom=sample_repetitions) 


#accompaniment = VarArray(size=number_of_samples_per_track, dom=get_domain_from_track_role(df, "accompaniment")) 
accompaniment = VarArray(size=[number_of_samples_per_track, len(accompaniment_ids)], dom={0, 1})
#accompaniment_measures = VarArray(size=[number_of_samples_per_track, len(accompaniment_ids)], dom=num_measures) 
accompaniment_repetitions = VarArray(size=[number_of_samples_per_track, len(accompaniment_ids)], dom=sample_repetitions) 

#riff = VarArray(size=number_of_samples_per_track, dom=get_domain_from_track_role(df, "riff")) 
riff = VarArray(size=[number_of_samples_per_track, len(riff_ids)], dom={0, 1})
#riff_measures = VarArray(size=[number_of_samples_per_track, len(riff_ids)], dom=num_measures) 
riff_repetitions = VarArray(size=[number_of_samples_per_track, len(riff_ids)], dom=sample_repetitions) 

#pad = VarArray(size=number_of_samples_per_track, dom=get_domain_from_track_role(df, "pad")) 
pad = VarArray(size=[number_of_samples_per_track, len(pad_ids)], dom={0, 1})
#pad_measures = VarArray(size=[number_of_samples_per_track, len(pad_ids)], dom=num_measures) 
pad_repetitions = VarArray(size=[number_of_samples_per_track, len(pad_ids)], dom=sample_repetitions) 

#sub_melody = VarArray(size=number_of_samples_per_track, dom=get_domain_from_track_role(df, "sub_melody"))
sub_melody = VarArray(size=[number_of_samples_per_track, len(sub_melody_ids)], dom={0, 1}) 
#sub_melody_measures = VarArray(size=[number_of_samples_per_track, len(sub_melody_ids)], dom=num_measures) 
sub_melody_repetitions = VarArray(size=[number_of_samples_per_track, len(sub_melody_ids)], dom=sample_repetitions) 

#bass = VarArray(size=number_of_samples_per_track, dom=get_domain_from_track_role(df, "bass")) 
bass = VarArray(size=[number_of_samples_per_track, len(bass_ids)], dom={0, 1}) 
#bass_measures = VarArray(size=[number_of_samples_per_track, len(bass_ids)], dom=num_measures) 
bass_repetitions = VarArray(size=[number_of_samples_per_track, len(bass_ids)], dom=sample_repetitions) 

#inter_track_distance = VarArray(size=number_of_samples_per_track, dom=[tpl[3] for tpl in same_duration_with_distance]) 

In [61]:
def exactly_one_sample_per_track_segment(vars):
    return [Sum(vars[i]) == 1 for i in range(number_of_samples_per_track)]

def bind_num_measures_bar_with_num_measures(vars, num_measures):
    return [vars[segment][commu_id_index] == num_measures[commu_id_index] for segment in range(number_of_samples_per_track) for commu_id_index in range(len(num_measures))],

def get_sum_of_measures_for_a_segment(ids, measures, repetitions, segment):
    return Sum([ids[segment][commu_id_index] * measures[commu_id_index] * repetitions[segment][commu_id_index] for commu_id_index in range(len(ids))])

def get_instruments_of_segments(ids, instrument_commu):
    return [Sum([ids[segment][commu_id_index] * instrument_commu[commu_id_index] for commu_id_index in range(len(ids))]) for segment in range(number_of_samples_per_track)]

unpost(ALL)
satisfy(
    
    # Exactly 1 sample is chose for each track segment (possibly the padding)
    exactly_one_sample_per_track_segment(main_melody),
    exactly_one_sample_per_track_segment(accompaniment),
    exactly_one_sample_per_track_segment(riff),
    exactly_one_sample_per_track_segment(bass),
    exactly_one_sample_per_track_segment(sub_melody),
    exactly_one_sample_per_track_segment(pad),
    
    # Matching the number of measures in each track segment
    [AllEqual(
        get_sum_of_measures_for_a_segment(main_melody, main_melody_num_measures, main_melody_repetitions, segment),
        get_sum_of_measures_for_a_segment(accompaniment, accompaniment_num_measures, accompaniment_repetitions, segment),
        get_sum_of_measures_for_a_segment(riff, riff_num_measures, riff_repetitions, segment),
        get_sum_of_measures_for_a_segment(pad, pad_num_measures, pad_repetitions, segment),
        get_sum_of_measures_for_a_segment(sub_melody, sub_melody_num_measures, sub_melody_repetitions, segment),
        get_sum_of_measures_for_a_segment(bass, bass_num_measures, bass_repetitions, segment))
        for segment in range(number_of_samples_per_track)], 
    
    # Matching instruments on the same track
    AllEqual(get_instruments_of_segments(main_melody, main_melody_instruments)),
    AllEqual(get_instruments_of_segments(accompaniment, accompaniment_instruments)),
    AllEqual(get_instruments_of_segments(riff, riff_instruments)),
    AllEqual(get_instruments_of_segments(pad, pad_instruments)),
    AllEqual(get_instruments_of_segments(sub_melody, sub_melody_instruments)),
    AllEqual(get_instruments_of_segments(bass, bass_instruments)),
    
    
    # Matching the number of measures in each track segment
    #[AllEqual(
    #    main_melody_measures[i] * main_melody_repetitions[i], 
    #    accompaniment_measures[i] * accompaniment_repetitions[i], 
    #    riff_measures[i] * riff_repetitions[i], 
    #    pad_measures[i] * pad_repetitions[i], 
    #    sub_melody_measures[i] * sub_melody_repetitions[i],
    #    bass_measures[i] * bass_repetitions[i]) for i in range(number_of_samples_per_track)],
    
    # Per continuare lo stesso sample sulla stessa traccia
    # main_melody[0] == main_melody[1],
    #[(main_melody[i], accompaniment[i], riff[i]) in same_duration for i in range(number_of_samples_per_track)]
    #[(main_melody[i], accompaniment[i], riff[i], inter_track_distance[i]) in same_duration_with_distance for i in range(number_of_samples_per_track)]
    #[(main_melody[i], main_melody_measures[i]) in get_num_measures_correspondence(df, "main_melody")  for i in range(number_of_samples_per_track)],
    #[(accompaniment[i], accompaniment_measures[i]) in get_num_measures_correspondence(df, "accompaniment") for i in range(number_of_samples_per_track)],
    #[(riff[i], riff_measures[i]) in get_num_measures_correspondence(df, "riff") for i in range(number_of_samples_per_track)],
    
)

sum(list:[main_melody[0][0], main_melody[0][1], main_melody[0][2], main_melody[0][3], main_melody[0][4], main_melody[0][5], main_melody[0][6], main_melody[0][7], main_melody[0][8], main_melody[0][9], main_melody[0][10], main_melody[0][11], main_melody[0][12], main_melody[0][13], main_melody[0][14], main_melody[0][15], main_melody[0][16], main_melody[0][17], main_melody[0][18], main_melody[0][19], main_melody[0][20], main_melody[0][21], main_melody[0][22], main_melody[0][23], main_melody[0][24], main_melody[0][25], main_melody[0][26], main_melody[0][27], main_melody[0][28], main_melody[0][29], main_melody[0][30], main_melody[0][31], main_melody[0][32], main_melody[0][33], main_melody[0][34], main_melody[0][35], main_melody[0][36], main_melody[0][37], main_melody[0][38], main_melody[0][39], main_melody[0][40], main_melody[0][41], main_melody[0][42], main_melody[0][43], main_melody[0][44], main_melody[0][45], main_melody[0][46], main_melody[0][47], main_melody[0][48], main_melody[0][49], 

In [16]:
#minimize(
#    Sum(main_melody_repetitions, accompaniment_repetitions, riff_repetitions, pad_repetitions, bass_repetitions, sub_melody_repetitions)
#)

In [17]:
solver(ACE)

In [64]:
#solve()
#solve(sols=ALL)
solve(sols=100)

<TypeStatus.SAT: 237>

In [72]:
def print_track(track_ids):
    track_string = ""
    for index, sample in enumerate(track_ids):
        track_string += f"{sample[0]} x {sample[1]} "
    return track_string
    
def to_commu_id(ids, commu_ids, repetitions):
    result = []
    for segment in range(number_of_samples_per_track):
        for i in range(len(ids[segment])):
            if(ids[segment][i]>0):
                result.append((commu_ids[i], repetitions[segment][i]))
    return result

for sol_number in range(n_solutions()):    
    print("Solution n ", sol_number)
    print("Main melody: ", print_track(to_commu_id(values(main_melody, sol=sol_number), main_melody_ids, values(main_melody_repetitions, sol=sol_number))))
    print("Accompaniment: ", print_track(to_commu_id(values(accompaniment, sol=sol_number), accompaniment_ids, values(accompaniment_repetitions, sol=sol_number))))
    print("Riff: ", print_track(to_commu_id(values(riff, sol=sol_number), riff_ids, values(riff_repetitions, sol=sol_number))))
    print("Pad: ", print_track(to_commu_id(values(pad, sol=sol_number), pad_ids, values(pad_repetitions, sol=sol_number))))
    print("Sub Melody: ", print_track(to_commu_id(values(sub_melody, sol=sol_number), sub_melody_ids, values(sub_melody_repetitions, sol=sol_number))))
    print("Bass: ", print_track(to_commu_id(values(bass, sol=sol_number), bass_ids, values(bass_repetitions, sol=sol_number))))
    

Solution n  0
Main melody:  commu11138 x * commu11138 x * commu11138 x * commu11138 x * 
Accompaniment:  commu11143 x * commu11143 x * commu11143 x * commu11143 x * 
Riff:  commu11141 x * commu11141 x * commu11141 x * commu11141 x * 
Pad:  commu11140 x * commu11140 x * commu11140 x * commu11140 x * 
Sub Melody:  commu11132 x * commu11132 x * commu11132 x * commu11132 x * 
Bass:  commu11144 x * commu11144 x * commu11144 x * commu11144 x * 
Solution n  1
Main melody:  commu11138 x * commu11138 x * commu11138 x * commu11138 x * 
Accompaniment:  commu11143 x * commu11143 x * commu11143 x * commu11143 x * 
Riff:  commu11141 x * commu11141 x * commu11141 x * commu11141 x * 
Pad:  commu11140 x * commu11140 x * commu11140 x * commu11140 x * 
Sub Melody:  commu11132 x * commu11132 x * commu11132 x * commu11132 x * 
Bass:  commu11144 x * commu11144 x * commu11144 x * commu11144 x * 
Solution n  2
Main melody:  commu11138 x * commu11138 x * commu11138 x * commu11138 x * 
Accompaniment:  commu1114